In [4]:
%%capture
!pip install python-dotenv
!pip install openai
!pip install requests
!pip install streamlit

## Setting the enviroment

In [5]:
from dotenv import load_dotenv
import os
from openai import OpenAI


load_dotenv()
os.getenv('OPENAI_API_KEY')
client = OpenAI()

## FInetuning the Model

In [3]:

train_file = client.files.create(
  file=open("video_to_json_train.jsonl", "rb"),
  purpose="fine-tune"
)
validation_file = client.files.create(
  file=open("video_to_json_validation.jsonl", "rb"),
  purpose="fine-tune"
)

response=client.fine_tuning.jobs.create(
  training_file=train_file.id,
  validation_file=validation_file.id,
  model="gpt-3.5-turbo",
  hyperparameters={
    'n_epochs': 4, 'batch_size': 1, 'learning_rate_multiplier': 2
  }
)
job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-zu3bdEm6DYYxcHXALoNvoTBo
Status: validating_files


In [6]:
job_id="ftjob-zu3bdEm6DYYxcHXALoNvoTBo"
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-zu3bdEm6DYYxcHXALoNvoTBo
Validating training file: file-yMNENRImt0Tz4sBm40ytxywp and validation file: file-z3wO8ITyBhEusHMw4ugKy3es


## FineTuned Model

In [7]:
import json
# response = client.fine_tuning.jobs.retrieve(job_id)
# fine_tuned_model_id = response.fine_tuned_model
completion = client.chat.completions.create( 
model="ft:gpt-3.5-turbo-0125:fast-nuces::9hBgC0oI",
  messages=[
  {
    "role": "system",
    "content": "Given a video description, provide the following fields in a JSON dict: \"output_format\" (string), \"duration\" (string), \"width\" (integer), \"height\" (integer), and \"elements\" (array of dicts with fields: \"type\", \"track\", \"time\", \"duration\", \"fill_color\", \"text\", \"font_family\")."
  },
  {
    "role": "user",
    "content": "Make JSON for a video that is 854x480 and 14 seconds long, with the text 'Please Wait' in magenta on a white background appear for 2 second, text\" Welcome to My Site\" in blue color for 4 second, text \"these are are the points to be disscussed\" in white for remainig second"

  }
  ]
)
try:
  json_data = json.loads(completion.choices[0].message.content)
  print(json_data)
except:
   print(completion.choices[0].message.content)

{'output_format': 'mp4', 'duration': '14 s', 'width': 854, 'height': 480, 'elements': [{'type': 'text', 'track': 1, 'time': '0 s', 'duration': '2 s', 'fill_color': '#ff00ff', 'text': 'Please Wait', 'font_family': 'Arial'}, {'type': 'text', 'track': 1, 'time': '2 s', 'duration': '4 s', 'fill_color': '#0000ff', 'text': 'Welcome to My Site', 'font_family': 'Arial'}, {'type': 'text', 'track': 1, 'time': '6 s', 'duration': '8 s', 'fill_color': '#ffffff', 'text': 'these are the points to be disscussed', 'font_family': 'Arial'}]}


## Base Model

In [8]:
completion = client.chat.completions.create(
 model="gpt-3.5-turbo",
  messages=[
  {
    "role": "system",
    "content": "Given a video description, provide the following fields in a JSON dict: \"output_format\" (string), \"duration\" (string), \"width\" (integer), \"height\" (integer), and \"elements\" (array of dicts with fields: \"type\", \"track\", \"time\", \"duration\", \"fill_color\", \"text\", \"font_family\")."
  },
  {
    "role": "user",
    "content": "Make JSON for a video that is 1280x720 and 22 seconds long, with the text 'Event Starts Soon' in cyan on a dark blue background."
  }
  ]
)

print(completion.choices[0].message.content)
import json
json_data = json.loads(completion.choices[0].message.content)

{
    "output_format": "mp4",
    "duration": "00:00:22",
    "width": 1280,
    "height": 720,
    "elements": [
        {
            "type": "text",
            "track": "video",
            "time": "00:00:00",
            "duration": "00:00:22",
            "fill_color": "cyan",
            "text": "Event Starts Soon",
            "font_family": "Arial"
        },
        {
            "type": "background",
            "track": "video",
            "time": "00:00:00",
            "duration": "00:00:22",
            "fill_color": "darkblue"
        }
    ]
}


## cost of finetuning?
For detailed pricing on training costs, as well as input and output costs for a deployed fine-tuned model, visit our pricing page. Note that we don't charge for tokens used for training validation. To estimate the cost of a specific fine-tuning training job, use the following formula:

    (base training cost per 1M input tokens ÷ 1M) × number of tokens in the input file × number of epochs trained

For a training file with 100,000 tokens trained over 3 epochs, the expected cost would be ~$2.40 USD with gpt-3.5-turbo-0125.
## where it is deployed?
it is managed by open ai servers . Sometimes it may cost more to get inference from finetuned model than the base model
## model expiry date
You can use it as long as you have membership
## add generate from base model as well in streamlit app



In [9]:
import requests

# Your API key
api_key = 'f5896694eed3459c874114716240507'

# The endpoint for current weather data
endpoint = 'http://api.weatherapi.com/v1/current.json'

# Parameters for the request
params = {
    'key': api_key,
    'q': 'London',  # Location query (e.g., city name, latitude/longitude, zip code, etc.)
    'aqi': 'no'     # Air Quality Index (optional, can be 'yes' or 'no')
}

# Sending the request
response = requests.get(endpoint, params=params)

# Checking if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Print the current weather data
    location_data = data['location']
    current_data = data['current']
    weather_info = (
            f"Location: {location_data['name']}, {location_data['region']}, {location_data['country']}\n"
            f"Local Time: {location_data['localtime']}\n"
            f"Temperature: {current_data['temp_c']}°C ({current_data['temp_f']}°F)\n"
            f"Condition: {current_data['condition']['text']}\n"
            f"Wind: {current_data['wind_mph']} mph ({current_data['wind_kph']} kph), "
            f"Direction: {current_data['wind_dir']} ({current_data['wind_degree']}°)\n"
            f"Pressure: {current_data['pressure_mb']} mb ({current_data['pressure_in']} in)\n"
            f"Precipitation: {current_data['precip_mm']} mm ({current_data['precip_in']} in)\n"
            f"Humidity: {current_data['humidity']}%\n"
            f"Cloud Cover: {current_data['cloud']}%\n"
            f"Feels Like: {current_data['feelslike_c']}°C ({current_data['feelslike_f']}°F)\n"
            f"Dew Point: {current_data['dewpoint_c']}°C ({current_data['dewpoint_f']}°F)\n"
            f"Visibility: {current_data['vis_km']} km ({current_data['vis_miles']} miles)\n"
            f"UV Index: {current_data['uv']}\n"
            f"Gust: {current_data['gust_mph']} mph ({current_data['gust_kph']} kph)"
        )
    
    print(data)
else:
    # Print the error message
    print(f"Error: {response.status_code}")
    print(response.json())


{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1720180349, 'localtime': '2024-07-05 12:52'}, 'current': {'last_updated_epoch': 1720179900, 'last_updated': '2024-07-05 12:45', 'temp_c': 19.3, 'temp_f': 66.7, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.34, 'precip_in': 0.01, 'humidity': 83, 'cloud': 25, 'feelslike_c': 19.3, 'feelslike_f': 66.7, 'windchill_c': 15.7, 'windchill_f': 60.2, 'heatindex_c': 15.7, 'heatindex_f': 60.2, 'dewpoint_c': 14.8, 'dewpoint_f': 58.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.0, 'gust_mph': 7.8, 'gust_kph': 12.6}}
